In [1]:
import torch
from transformers import HubertModel, BertModel

In [2]:
distillhubert = HubertModel.from_pretrained('ntu-spml/distilhubert') # or this: patrickvonplaten/distilhubert-timit

In [4]:
bert = BertModel.from_pretrained('google/bert_uncased_L-8_H-512_A-8')

Downloading: 100%|██████████| 383/383 [00:00<00:00, 96.0kB/s]
Downloading: 100%|██████████| 159M/159M [00:21<00:00, 7.90MB/s] 
Some weights of the model checkpoint at google/bert_uncased_L-8_H-512_A-8 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

In [5]:
count_parameters = lambda model : {'requires_grad':sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6,
                                   'does_not_require_grad':sum(p.numel() for p in model.parameters() if not p.requires_grad)/1e6}

count_parameters(bert)

{'requires_grad': 41.373184, 'does_not_require_grad': 0.0}

In [4]:
projector = distillhubert.feature_projection
encoder = distillhubert.encoder

In [21]:
count_parameters(encoder)

{'requires_grad': 18.896768, 'does_not_require_grad': 0.0}

In [6]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input = torch.randn((16, 128, 512)) #, device=device)

# projector.to(device)
# encoder.to(device)

with torch.no_grad():
    outputs = projector(input)
    outputs = encoder(outputs)

print(outputs.last_hidden_state.size())

torch.Size([16, 128, 768])


In [8]:
print(outputs[0])

tensor([[[ 3.5730e-01,  6.8287e-02, -6.5710e-02,  ...,  3.1320e-01,
           2.7473e-01, -4.0593e-01],
         [ 5.9186e-01,  7.4721e-02, -1.1976e-01,  ...,  1.5153e-01,
           4.7239e-01,  8.6874e-01],
         [ 3.4321e-01, -2.4234e-01, -8.3484e-02,  ...,  2.9233e-02,
           2.7845e-01, -5.6123e-01],
         ...,
         [ 4.5318e-01, -1.3775e-01, -4.3091e-01,  ...,  4.5748e-02,
           1.9874e-01, -4.8692e-01],
         [ 2.3670e-01,  1.3775e-01, -2.8171e-02,  ...,  2.7102e-01,
           2.6036e-01, -6.8628e-02],
         [ 4.1774e-01, -9.2515e-02,  1.3279e-01,  ...,  3.2909e-01,
           2.2394e-01,  1.1137e-01]],

        [[ 5.0577e-01, -8.7658e-03, -7.6905e-03,  ...,  8.1075e-02,
           3.8099e-01,  2.0468e-01],
         [ 3.4292e-01, -1.1005e-01, -1.7447e-01,  ...,  1.2369e-01,
           3.1858e-01, -8.0807e-01],
         [ 1.7262e-01, -1.6458e-01, -5.7281e-02,  ..., -2.5144e-03,
           1.3199e-01, -1.0956e-01],
         ...,
         [ 2.8035e-01,  1

In [23]:
torch.cuda.empty_cache()